In [7]:
import numpy as np
import pandas as pd

import nested_utilities as nutil
import nested_predict as npred

from keras.models import load_model

In [8]:
model_to_use = './NO_SYNC/nested/model_output_2018-03-18_21/checkpoints/Incv3_nested_run_4_FT_ft_20-0.63.hdf5'
model = load_model(model_to_use)

In [9]:
# build a catalogue of the test images 
test_catalogue = nutil.build_catalogue('./data/test_data/')


# define list of interior rooms (/scenes) and use to create catalogue 
# of interior scenes (only) in test set.
# NOTE it is important this list is in strict alphabetical order,
# so that it matches keras model output labels (0-10)
# reset_index to set to continous count (needed for merge with predictions below),
# and drop surplus 'index' column
interior_rooms = ['bathroom',
                  'bedroom',
                  'conservatory', 
                  'diningroom',
                  'empty',
                  'entrance',
                  'graphic', 
                  'kitchen',
                  'livingroom', 
                  'misc_int', 
                  'study']

interior_test_cat = test_catalogue.loc[test_catalogue['room'].isin(interior_rooms)].copy()
interior_test_cat.reset_index(inplace=True)
interior_test_cat.drop('index', axis=1, inplace=True)

In [10]:
pred_array = npred.predict_catalogue(interior_test_cat.iloc[0:25,:], model)

Starting prediction for 25 images
Predictions complete. 25 image predictions made


In [11]:
pred_df = npred.pred_to_df(pred_array, columns=interior_rooms)
pred_labels = npred.get_labels_and_probs(pred_df, num_labels=4)
pred_labels.head()

,label1,label2,label3,label4,label1_P,label2_P,label3_P,label4_P
0,bathroom,kitchen,empty,bedroom,0.999998,0.000001,7.169086e-08,1.372046e-08
1,bathroom,kitchen,diningroom,bedroom,0.999782,0.000217,9.364485e-07,5.479097e-07
2,bathroom,bedroom,kitchen,misc_int,0.999678,0.000118,1.028072e-04,3.945936e-05
3,bathroom,kitchen,bedroom,livingroom,0.999844,0.000082,5.523511e-05,7.788133e-06
4,bathroom,kitchen,diningroom,empty,0.999998,0.000002,1.329193e-09,3.304619e-10


In [12]:
interior_test_pred = pd.merge(interior_test_cat, pred_labels, how='left', 
                              left_index=True, right_index=True)

print(interior_test_pred.info())
interior_test_pred.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3859 entries, 0 to 3858
Data columns (total 12 columns):
id            3859 non-null object
room          3859 non-null object
filename      3859 non-null object
image_path    3859 non-null object
label1        25 non-null object
label2        25 non-null object
label3        25 non-null object
label4        25 non-null object
label1_P      25 non-null float64
label2_P      25 non-null float64
label3_P      25 non-null float64
label4_P      25 non-null float64
dtypes: float64(4), object(8)
memory usage: 361.9+ KB
None


,id,room,filename,image_path,label1,label2,label3,label4,label1_P,label2_P,label3_P,label4_P
0,003457d1fb62155b4531aba3e5b39f4d57bed9f9,bathroom,003457d1fb62155b4531aba3e5b39f4d57bed9f9.jpg,./data/test_data/bathroom/003457d1fb62155b4531...,bathroom,kitchen,empty,bedroom,0.999998,0.000001,7.169086e-08,1.372046e-08
1,0038cad4553a9922d24a70a4d77abb9d2c1261f9,bathroom,0038cad4553a9922d24a70a4d77abb9d2c1261f9.jpg,./data/test_data/bathroom/0038cad4553a9922d24a...,bathroom,kitchen,diningroom,bedroom,0.999782,0.000217,9.364485e-07,5.479097e-07
2,003f5052b875f64f4f5ce99948ae87f35d44b009,bathroom,003f5052b875f64f4f5ce99948ae87f35d44b009.jpg,./data/test_data/bathroom/003f5052b875f64f4f5c...,bathroom,bedroom,kitchen,misc_int,0.999678,0.000118,1.028072e-04,3.945936e-05
3,009e866e3eb114c3bc8579d780a4c333ddd38da9,bathroom,009e866e3eb114c3bc8579d780a4c333ddd38da9.jpg,./data/test_data/bathroom/009e866e3eb114c3bc85...,bathroom,kitchen,bedroom,livingroom,0.999844,0.000082,5.523511e-05,7.788133e-06
4,00da56e13e015b383effff3fe8560ae7fa7cb1f9,bathroom,00da56e13e015b383effff3fe8560ae7fa7cb1f9.jpg,./data/test_data/bathroom/00da56e13e015b383eff...,bathroom,kitchen,diningroom,empty,0.999998,0.000002,1.329193e-09,3.304619e-10
